In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
instalments = pd.read_csv("../../../../homecredit/installments_payments.csv")
application_train = pd.read_csv('../../../../homecredit/application_train.csv')
pos_cash= pd.read_csv("../../../../homecredit/POS_CASH_balance.csv")
previous_application = pd.read_csv("../../../../homecredit/previous_application.csv")
credit_card_balance= pd.read_csv("../../../../homecredit/credit_card_balance.csv")
bureau_avg= pd.read_csv("../../../../homecredit/bureau_average.csv")
bureau= pd.read_csv("../../../../homecredit/bureau.csv")

# Train

In [3]:
# List of columns to drop
columns_to_drop = [
    'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE','FLAG_EMAIL',
    'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
    'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG',
    'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
    'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG',
    'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
    'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
    'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
    'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
    'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
    'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
    'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
    'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
    'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
    'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK'
]

application_train= application_train.drop(columns=columns_to_drop)

In [4]:
application_train['365243DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].apply(lambda x: 1 if x == 365243 else 0)
application_train['DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].replace(365243, np.nan)

In [5]:
# change DAYS_BIRTH to AGE
application_train["DAYS_BIRTH"] = application_train["DAYS_BIRTH"] / 365
application_train["DAYS_BIRTH"] = application_train["DAYS_BIRTH"].astype(int).abs()
application_train = application_train.rename(columns={"DAYS_BIRTH": "AGE"})

# change DAYS_EMPLOYED to JOB_TENURE
application_train["DAYS_EMPLOYED"] = application_train["DAYS_EMPLOYED"] / 365
application_train["DAYS_EMPLOYED"] = application_train["DAYS_EMPLOYED"].round(1).abs()
application_train = application_train.rename(columns={"DAYS_EMPLOYED": "JOB_TENURE"})

#處理極端值
application_train['AMT_INCOME_TOTAL'] = application_train['AMT_INCOME_TOTAL'].replace(100000000, np.nan)
application_train[[
    'DAYS_REGISTRATION','DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE']] = application_train[[
    'DAYS_REGISTRATION','DAYS_ID_PUBLISH','DAYS_LAST_PHONE_CHANGE']].abs()


application_train['REGION_RATING_CLIENT_W_CITY'] = application_train['REGION_RATING_CLIENT_W_CITY'].replace(-1, 0)


# (11) Replace values greater than 100 in 'AMT_REQ_CREDIT_BUREAU_QRT' column with np.nan
application_train['AMT_REQ_CREDIT_BUREAU_QRT'] = application_train['AMT_REQ_CREDIT_BUREAU_QRT'].apply(lambda x: np.nan if x > 100 else x)

#缺失值補0
application_train[[
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR','OWN_CAR_AGE','EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3','AMT_REQ_CREDIT_BUREAU_QRT','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']] = application_train[[
    'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR','OWN_CAR_AGE','EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3','AMT_REQ_CREDIT_BUREAU_QRT','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']].fillna(0)

#缺失值補平均數
columns_to_fill_mean = ['AMT_ANNUITY','AMT_GOODS_PRICE',"JOB_TENURE",'DAYS_LAST_PHONE_CHANGE','CNT_FAM_MEMBERS']  
mean_values = {}
for col in columns_to_fill_mean:
    mean_values[col] = application_train[col].mean()
    application_train.loc[application_train[col].isna(), col] = mean_values[col]


#缺失值補隨機值
columns_to_fill_random = ['NAME_TYPE_SUITE', 'OCCUPATION_TYPE']

for col in columns_to_fill_random:
    random_values = np.random.choice(application_train[col].dropna(), size=application_train[col].isna().sum())
    application_train.loc[application_train[col].isna(), col] = random_values


In [6]:
print(application_train.isna().sum()/application_train.shape[0])
application_train.shape

SK_ID_CURR                     0.0
TARGET                         0.0
NAME_CONTRACT_TYPE             0.0
CODE_GENDER                    0.0
FLAG_OWN_CAR                   0.0
FLAG_OWN_REALTY                0.0
CNT_CHILDREN                   0.0
AMT_INCOME_TOTAL               0.0
AMT_CREDIT                     0.0
AMT_ANNUITY                    0.0
AMT_GOODS_PRICE                0.0
NAME_TYPE_SUITE                0.0
NAME_INCOME_TYPE               0.0
NAME_EDUCATION_TYPE            0.0
NAME_FAMILY_STATUS             0.0
NAME_HOUSING_TYPE              0.0
REGION_POPULATION_RELATIVE     0.0
AGE                            0.0
JOB_TENURE                     0.0
DAYS_REGISTRATION              0.0
DAYS_ID_PUBLISH                0.0
OWN_CAR_AGE                    0.0
OCCUPATION_TYPE                0.0
CNT_FAM_MEMBERS                0.0
REGION_RATING_CLIENT           0.0
REGION_RATING_CLIENT_W_CITY    0.0
WEEKDAY_APPR_PROCESS_START     0.0
HOUR_APPR_PROCESS_START        0.0
ORGANIZATION_TYPE   

(307511, 39)

# previous_application

In [7]:
columns_to_drop = ['WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 
                   'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 
                   'CODE_REJECT_REASON', 'NAME_CLIENT_TYPE', 'NAME_PORTFOLIO', 'AMT_DOWN_PAYMENT',
       'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA',
       'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL']

previous_application = previous_application.drop(columns=columns_to_drop)




previous_application = previous_application.groupby(['SK_ID_CURR', 'NAME_CONTRACT_TYPE']).mean().reset_index()

print(previous_application.head())

   SK_ID_CURR NAME_CONTRACT_TYPE  SK_ID_PREV  AMT_ANNUITY  AMT_APPLICATION  \
0      100001     Consumer loans   1369693.0    3951.0000         24835.50   
1      100002     Consumer loans   1038818.0    9251.7750        179055.00   
2      100003         Cash loans   1810518.0   98356.9950        900000.00   
3      100003     Consumer loans   2516466.5   35652.4875        203154.75   
4      100004     Consumer loans   1564014.0    5357.2500         24282.00   

   AMT_CREDIT  AMT_GOODS_PRICE  
0     23787.0         24835.50  
1    179055.0        179055.00  
2   1035882.0        900000.00  
3    208345.5        203154.75  
4     20106.0         24282.00  


C:\Users\User\AppData\Local\Temp\ipykernel_9964\3475350568.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  previous_application = previous_application.groupby(['SK_ID_CURR', 'NAME_CONTRACT_TYPE']).mean().reset_index()


In [8]:
previous_application.shape

(637901, 7)

# ⭐⭐⭐這以下+三個合表

In [9]:
prev_consumer_loans =previous_application [previous_application ['NAME_CONTRACT_TYPE'] == 'Consumer loans']

# 重新命名列
prev_consumer_loans.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_CONSUMER_LOANS_MEAN',
                              'AMT_APPLICATION': 'AMT_APPLICATION_CONSUMER_LOANS_MEAN',
                              'AMT_CREDIT': 'AMT_CREDIT_CONSUMER_LOANS_MEAN',
                              'AMT_GOODS_PRICE': 'AMT_GOODS_PRICE_CONSUMER_LOANS_MEAN'},
                     inplace=True)

#KNN補缺失值
cols_to_fill_na = ['AMT_ANNUITY_CONSUMER_LOANS_MEAN', 'AMT_APPLICATION_CONSUMER_LOANS_MEAN', 'AMT_CREDIT_CONSUMER_LOANS_MEAN','AMT_GOODS_PRICE_CONSUMER_LOANS_MEAN']
data_to_fill = prev_consumer_loans[cols_to_fill_na]
knn_imputer = KNNImputer(n_neighbors=3)  # 选择适当的 n_neighbors 值
filled_data = knn_imputer.fit_transform(data_to_fill)
prev_consumer_loans[cols_to_fill_na] = filled_data




prev_consumer_loans.isna().sum()


C:\Users\User\AppData\Local\Temp\ipykernel_9964\676660153.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_consumer_loans.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_CONSUMER_LOANS_MEAN',
C:\Users\User\AppData\Local\Temp\ipykernel_9964\676660153.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_consumer_loans[cols_to_fill_na] = filled_data


SK_ID_CURR                             0
NAME_CONTRACT_TYPE                     0
SK_ID_PREV                             0
AMT_ANNUITY_CONSUMER_LOANS_MEAN        0
AMT_APPLICATION_CONSUMER_LOANS_MEAN    0
AMT_CREDIT_CONSUMER_LOANS_MEAN         0
AMT_GOODS_PRICE_CONSUMER_LOANS_MEAN    0
dtype: int64

In [10]:
#9min
prev_cash_loans = previous_application[previous_application['NAME_CONTRACT_TYPE'] == 'Cash loans']

# 重新命名列
prev_cash_loans.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_CASH_LOANS_MEAN',
                              'AMT_APPLICATION': 'AMT_APPLICATION_CASH_LOANS_MEAN',
                              'AMT_CREDIT': 'AMT_CREDIT_CASH_LOANS_MEAN',
                              'AMT_GOODS_PRICE': 'AMT_GOODS_PRICE_CASH_LOANS_MEAN'},
                     inplace=True)


#KNN補缺失值
cols_to_fill_na = ['AMT_ANNUITY_CASH_LOANS_MEAN', 'AMT_APPLICATION_CASH_LOANS_MEAN', 'AMT_CREDIT_CASH_LOANS_MEAN','AMT_GOODS_PRICE_CASH_LOANS_MEAN']
data_to_fill = prev_cash_loans[cols_to_fill_na]
knn_imputer = KNNImputer(n_neighbors=3) 
filled_data = knn_imputer.fit_transform(data_to_fill)
prev_cash_loans[cols_to_fill_na] = filled_data



prev_cash_loans.isna().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_9964\4190160715.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_cash_loans.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_CASH_LOANS_MEAN',
C:\Users\User\AppData\Local\Temp\ipykernel_9964\4190160715.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_cash_loans[cols_to_fill_na] = filled_data


SK_ID_CURR                         0
NAME_CONTRACT_TYPE                 0
SK_ID_PREV                         0
AMT_ANNUITY_CASH_LOANS_MEAN        0
AMT_APPLICATION_CASH_LOANS_MEAN    0
AMT_CREDIT_CASH_LOANS_MEAN         0
AMT_GOODS_PRICE_CASH_LOANS_MEAN    0
dtype: int64

In [11]:
#5min
prev_revolving_loans = previous_application[previous_application['NAME_CONTRACT_TYPE'] == 'Revolving loans']

# 重新命名列
prev_revolving_loans.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_REVOLVING_LOANS_MEAN',
                              'AMT_APPLICATION': 'AMT_APPLICATION_REVOLVING_LOANS_MEAN',
                              'AMT_CREDIT': 'AMT_CREDIT_REVOLVING_LOANS_MEAN',
                              'AMT_GOODS_PRICE': 'AMT_GOODS_PRICE_REVOLVING_LOANS_MEAN'},
                     inplace=True)


#KNN補缺失值
cols_to_fill_na = ['AMT_ANNUITY_REVOLVING_LOANS_MEAN', 'AMT_APPLICATION_REVOLVING_LOANS_MEAN', 'AMT_CREDIT_REVOLVING_LOANS_MEAN','AMT_GOODS_PRICE_REVOLVING_LOANS_MEAN']
data_to_fill = prev_revolving_loans[cols_to_fill_na]
knn_imputer = KNNImputer(n_neighbors=3)  
filled_data = knn_imputer.fit_transform(data_to_fill)
prev_revolving_loans[cols_to_fill_na] = filled_data



prev_revolving_loans.isna().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_9964\1268001576.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_revolving_loans.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_REVOLVING_LOANS_MEAN',
C:\Users\User\AppData\Local\Temp\ipykernel_9964\1268001576.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_revolving_loans[cols_to_fill_na] = filled_data


SK_ID_CURR                              0
NAME_CONTRACT_TYPE                      0
SK_ID_PREV                              0
AMT_ANNUITY_REVOLVING_LOANS_MEAN        0
AMT_APPLICATION_REVOLVING_LOANS_MEAN    0
AMT_CREDIT_REVOLVING_LOANS_MEAN         0
AMT_GOODS_PRICE_REVOLVING_LOANS_MEAN    0
dtype: int64

# Credit_Card

In [12]:
#確認負數   ETA:3m
real_pay=credit_card_balance['AMT_PAYMENT_CURRENT']
#print((real_pay < 0).sum())
should_pay=credit_card_balance['AMT_TOTAL_RECEIVABLE']
#print((should_pay < 0).sum())
min_should_pay=credit_card_balance['AMT_INST_MIN_REGULARITY']
#print((min_should_pay < 0).sum())

###############################################
#real_pay 遺失值約20%，10%是0，負數有0個
#10%為0不處理 我認為合裡上期沒有付款而已，沒有負數是正常值
#遺失值我想用<平均數去填>
real_pay_mean_value = real_pay.mean()
real_pay.fillna(real_pay_mean_value, inplace=True)

#檢測是否補值完

#print(real_pay.isnull().sum())
#print((real_pay==0).sum())

###############################################
#should_pay 沒有遺失值，55%是0，負數有109330個
#負數有10萬筆我視為寫錯將<負數轉為正數>
#55%是0 不太合理 應付款項不應該為0 <用平均數去補值>

#轉正數
should_pay = should_pay.abs()

#確認平均數、中位數
#print(should_pay.mean()) 
#print(should_pay.median())

#檢測補0前補0後
#print((should_pay==0).sum())
should_pay[should_pay == 0]=should_pay.mean()
#print((should_pay==0).sum())

###############################################
#min_should_pay 遺失值8%，50%是0，負數有0個
#55%是0 不太合理 最小應付款項不應該為0  <用平均數補值>
#我的做法是先用平均數補0 再去把新的資料平均數之後補遺失值

#檢測補0前補0後
#print((min_should_pay==0).sum())
min_should_pay[min_should_pay == 0]=min_should_pay.mean()
#print((min_should_pay==0).sum())

#檢測補遺失值前後
#print(min_should_pay.isnull().sum())
min_should_pay.fillna(min_should_pay.mean(), inplace=True)
#print(min_should_pay.isnull().sum())


credit_card_balance['creditcard_high'] = (real_pay>=should_pay)==True
credit_card_balance['creditcard_normal'] = ((real_pay<should_pay)&(real_pay>=min_should_pay))==True
credit_card_balance['creditcard_low'] = (real_pay<min_should_pay)==True

#檢測單熱是否做完
"""
print(credit_card_balance['creditcard_high'].head())
print('-'*80)
print(credit_card_balance['creditcard_normal'].head())
print('-'*80)
print(credit_card_balance['creditcard_low'].head())
print('-'*80)
normal_true = credit_card_balance['creditcard_high'].sum()
print(normal_true)
print(normal_true/credit_card_balance.shape[0])
nice_true = credit_card_balance['creditcard_normal'].sum()
print(nice_true)
print(nice_true/credit_card_balance.shape[0])
bad_true = credit_card_balance['creditcard_low'].sum()
print(bad_true)
print(bad_true/credit_card_balance.shape[0])
"""
def majority_vote(series):
    counts = series.value_counts()
    total_count = counts.sum()
    majority_value = counts.idxmax()
    majority_count = counts.max()
    return majority_value


result_high = credit_card_balance.groupby('SK_ID_CURR').apply(lambda group: majority_vote(group['creditcard_high'])).reset_index(drop=True)
result_normal = credit_card_balance.groupby('SK_ID_CURR').apply(lambda group: majority_vote(group['creditcard_normal'])).reset_index(drop=True)
result_low = credit_card_balance.groupby('SK_ID_CURR').apply(lambda group: majority_vote(group['creditcard_low'])).reset_index(drop=True)

credit_card_balance = pd.DataFrame({
    'SK_ID_CURR': credit_card_balance['SK_ID_CURR'].unique(),
    'creditcard_high': result_high,
    'creditcard_normal': result_normal,
    'creditcard_low': result_low
})

credit_card_balance = credit_card_balance.sort_values(by=['SK_ID_CURR'])
credit_card_balance=credit_card_balance.reset_index(drop=True)
credit_card_balance

C:\Users\User\AppData\Local\Temp\ipykernel_9964\1662834490.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_should_pay[min_should_pay == 0]=min_should_pay.mean()


,SK_ID_CURR,creditcard_high,creditcard_normal,creditcard_low
0,100006,False,True,False
1,100011,False,True,False
2,100013,False,False,True
3,100021,False,False,True
4,100023,False,True,False
...,...,...,...,...
103553,456244,False,False,False
103554,456246,False,True,False
103555,456247,False,False,True
103556,456248,False,True,False


In [13]:
credit_card_balance['creditcard_high'] = credit_card_balance['creditcard_high'].replace({True: 1, False: 0})
credit_card_balance['creditcard_normal'] = credit_card_balance['creditcard_normal'].replace({True: 1, False: 0})
credit_card_balance['creditcard_low'] = credit_card_balance['creditcard_low'].replace({True: 1, False: 0})

In [14]:
credit_card_balance

,SK_ID_CURR,creditcard_high,creditcard_normal,creditcard_low
0,100006,0,1,0
1,100011,0,1,0
2,100013,0,0,1
3,100021,0,0,1
4,100023,0,1,0
...,...,...,...,...
103553,456244,0,0,0
103554,456246,0,1,0
103555,456247,0,0,1
103556,456248,0,1,0


# insalments

In [15]:
instalments = instalments.sort_values(by=['SK_ID_CURR','SK_ID_PREV','NUM_INSTALMENT_NUMBER']) 
instalments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
1478621,1369693,100001,1.0,1,-1709.0,-1715.0,3951.000,3951.000
2568722,1369693,100001,1.0,2,-1679.0,-1715.0,3951.000,3951.000
3458712,1369693,100001,1.0,3,-1649.0,-1660.0,3951.000,3951.000
2624024,1369693,100001,2.0,4,-1619.0,-1628.0,17397.900,17397.900
1761012,1851984,100001,1.0,2,-2916.0,-2916.0,3982.050,3982.050
...,...,...,...,...,...,...,...,...
12816277,2631384,456255,3.0,23,-96.0,-98.0,27489.690,27489.690
12164846,2631384,456255,4.0,24,-66.0,-76.0,308277.315,308277.315
12300604,2729207,456255,1.0,1,-469.0,-482.0,11514.555,11514.555
12567314,2729207,456255,1.0,2,-439.0,-455.0,11514.555,11514.555


In [16]:
# AMT_ENTRY_PAYMENT及AMT_PAYMENT兩個欄位空值的行數相同
both_columns_missing =(instalments[['DAYS_ENTRY_PAYMENT', 'AMT_PAYMENT']].isnull().all(axis=1)).sum()
# 將Payment空值的欄位填入應繳欄位的值 == 無遲繳也無早繳
instalments['DAYS_ENTRY_PAYMENT'].fillna(instalments['DAYS_INSTALMENT'], inplace=True)
instalments['AMT_PAYMENT'].fillna(instalments['AMT_INSTALMENT'], inplace=True)
instalments.isna().sum()

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

In [17]:
#overdue_DAY_mean 正數為遲繳(天數)，負數為提前繳(天數)
overdue_DAY_mean = instalments.DAYS_ENTRY_PAYMENT - instalments.DAYS_INSTALMENT

In [18]:
#overdue_AMT_mean 正數表示多繳，負數表示少繳
overdue_AMT_mean = instalments.AMT_PAYMENT -instalments.AMT_INSTALMENT

In [19]:
#計算每人的平均貸款期數
#每筆貸款的最大期數視為貸款總期數
#再依照人的ID計算貸款期數的平均
instalments_PREV_CNT = instalments.groupby('SK_ID_PREV').max()[['SK_ID_CURR','NUM_INSTALMENT_NUMBER']]
instalments_PERIOD = instalments_PREV_CNT.groupby('SK_ID_CURR').mean().reset_index()

In [20]:
#合併到indstalments表格
instalments['overdue_DAY_mean'] = overdue_DAY_mean
instalments['overdue_AMT_mean'] = overdue_AMT_mean
instalments = instalments.groupby("SK_ID_CURR").mean().reset_index()
#overdue_DAY 正數為遲繳(天數)，負數為提前繳(天數)
#overdue_AMT 正數表示多繳，負數表示少繳

In [21]:
#overdue_DAY 正數為遲繳(天數)，負數為提前繳(天數)
#overdue_AMT 正數表示多繳，負數表示少繳
instalments_PERIOD = instalments_PERIOD.rename(columns={'NUM_INSTALMENT_NUMBER':'instalments_PERIOD'})
instalments = pd.merge(instalments,instalments_PERIOD,on='SK_ID_CURR')

instalments = instalments[['SK_ID_CURR', 'overdue_DAY_mean', 'overdue_AMT_mean', 'instalments_PERIOD']].rename(
    columns={
    'overdue_DAY_mean': 'instalments_overdue_DAY_mean',
    'overdue_AMT_mean': 'instalments_overdue_AMT_mean'
})


print(instalments.head())
print(instalments.shape)

   SK_ID_CURR  instalments_overdue_DAY_mean  instalments_overdue_AMT_mean  \
0      100001                     -7.285714                           0.0   
1      100002                    -20.421053                           0.0   
2      100003                     -7.160000                           0.0   
3      100004                     -7.666667                           0.0   
4      100005                    -23.555556                           0.0   

   instalments_PERIOD  
0            4.000000  
1           19.000000  
2            8.333333  
3            3.000000  
4            9.000000  
(339587, 4)


# Bureau head


In [22]:
head = ['SK_ID_CURR', 'SIB', 'CA', 'CC', 'DC',
        'CDO', 'DCE', 'DEF', 'ACMO', 'CCP', 'ACS', 'ACSD', 'ACSL', 'ACSO', 'CT', 'DCU', 'AA' ]
br_df =bureau.copy(deep=True)
br_df.columns = head

print(br_df.head())

   SK_ID_CURR      SIB      CA          CC   DC  CDO     DCE    DEF     ACMO  \
0      215354  5714462  Closed  currency 1 -497    0  -153.0 -153.0      NaN   
1      215354  5714463  Active  currency 1 -208    0  1075.0    NaN      NaN   
2      215354  5714464  Active  currency 1 -203    0   528.0    NaN      NaN   
3      215354  5714465  Active  currency 1 -203    0     NaN    NaN      NaN   
4      215354  5714466  Active  currency 1 -629    0  1197.0    NaN  77674.5   

   CCP        ACS      ACSD  ACSL  ACSO               CT  DCU  AA  
0    0    91323.0       0.0   NaN   0.0  Consumer credit -131 NaN  
1    0   225000.0  171342.0   NaN   0.0      Credit card  -20 NaN  
2    0   464323.5       NaN   NaN   0.0  Consumer credit  -16 NaN  
3    0    90000.0       NaN   NaN   0.0      Credit card  -16 NaN  
4    0  2700000.0       NaN   NaN   0.0  Consumer credit  -21 NaN  


In [23]:
x=br_df[br_df.CA=='Active']
x[br_df.DEF<0]

C:\Users\User\AppData\Local\Temp\ipykernel_9964\2549190388.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x[br_df.DEF<0]


,SK_ID_CURR,SIB,CA,CC,DC,CDO,DCE,DEF,ACMO,CCP,ACS,ACSD,ACSL,ACSO,CT,DCU,AA
1872,310981,5716773,Active,currency 1,-403,0,-188.0,-403.0,0.0,0,70487.190,0.0,0.0,0.0,Consumer credit,-403,NaN
3549,187117,5718814,Active,currency 1,-239,0,126.0,-56.0,NaN,0,110250.000,0.0,NaN,0.0,Consumer credit,-32,NaN
4124,202848,5719466,Active,currency 1,-527,0,-41.0,-71.0,NaN,0,74475.000,0.0,NaN,0.0,Consumer credit,-53,NaN
4171,177829,5719521,Active,currency 1,-2740,0,-2008.0,-1195.0,0.0,0,103500.000,0.0,0.0,0.0,Consumer credit,-1195,4140.0
4685,156464,5720103,Active,currency 1,-430,0,-63.0,-63.0,NaN,0,285111.180,0.0,NaN,0.0,Consumer credit,-36,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711721,110734,5090013,Active,currency 1,-60,0,-30.0,-44.0,0.0,0,22500.000,0.0,0.0,0.0,Consumer credit,-26,NaN
1711736,410645,5095999,Active,currency 1,-2685,0,-1559.0,-1349.0,0.0,0,225000.000,0.0,0.0,0.0,Credit card,-1225,NaN
1712703,398889,5077908,Active,currency 1,-1092,0,-696.0,-1092.0,0.0,0,140313.735,0.0,0.0,0.0,Consumer credit,-1092,NaN
1713185,199456,5085410,Active,currency 1,-2472,0,-1366.0,-2409.0,0.0,0,45000.000,0.0,0.0,0.0,Credit card,-646,NaN


In [24]:
x=br_df[br_df.CA=='Closed']
x[br_df.DEF<0]

C:\Users\User\AppData\Local\Temp\ipykernel_9964\1369278286.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x[br_df.DEF<0]


,SK_ID_CURR,SIB,CA,CC,DC,CDO,DCE,DEF,ACMO,CCP,ACS,ACSD,ACSL,ACSO,CT,DCU,AA
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.0,0.0,0.0,Consumer credit,-840,NaN
11,162297,5714473,Closed,currency 1,-2456,0,-629.0,-825.0,NaN,0,675000.00,0.0,0.0,0.0,Consumer credit,-706,NaN
14,238881,5714482,Closed,currency 1,-318,0,-187.0,-187.0,NaN,0,0.00,0.0,0.0,0.0,Credit card,-185,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716422,375755,5057742,Closed,currency 1,-2648,0,31129.0,-189.0,NaN,0,202500.00,0.0,NaN,0.0,Credit card,-109,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [25]:
br_df[br_df.ACMO>0 ]

,SK_ID_CURR,SIB,CA,CC,DC,CDO,DCE,DEF,ACMO,CCP,ACS,ACSD,ACSL,ACSO,CT,DCU,AA
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.500,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.000,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,NaN
24,222183,5714495,Closed,currency 1,-2744,0,-2561.0,-2559.0,310.500,0,18157.50,NaN,NaN,0.0,Consumer credit,-2559,NaN
25,222183,5714496,Closed,currency 1,-1103,0,-7.0,-343.0,20493.270,0,675000.00,0.0,0.0,0.0,Consumer credit,-343,NaN
26,222183,5714497,Active,currency 1,-315,0,1512.0,NaN,88821.000,0,3709552.50,NaN,NaN,0.0,Car loan,-32,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716405,438109,5057605,Closed,currency 1,-2390,0,-2053.0,-2082.0,11317.590,0,112204.35,0.0,0.0,0.0,Consumer credit,-2082,NaN
1716413,288038,5057665,Closed,currency 1,-463,0,-36.0,-36.0,6863.715,0,110878.47,0.0,0.0,0.0,Consumer credit,-33,NaN
1716414,445222,5057676,Closed,currency 1,-1628,0,-1320.0,-1320.0,5693.625,0,57834.00,0.0,0.0,0.0,Consumer credit,-1320,NaN
1716419,352790,5057718,Closed,currency 1,-1808,0,-1596.0,-1625.0,8100.000,0,28248.84,0.0,0.0,0.0,Consumer credit,-1625,NaN


In [26]:
br_df[br_df.DCE<0 ]

,SK_ID_CURR,SIB,CA,CC,DC,CDO,DCE,DEF,ACMO,CCP,ACS,ACSD,ACSL,ACSO,CT,DCU,AA
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
7,162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,NaN
8,162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.70,0.0,0.0,0.0,Consumer credit,-840,NaN
9,162297,5714471,Active,currency 1,-1146,0,-484.0,NaN,0.0,0,4500.00,0.0,0.0,0.0,Credit card,-690,NaN
10,162297,5714472,Active,currency 1,-1146,0,-180.0,NaN,0.0,0,337500.00,0.0,0.0,0.0,Credit card,-690,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716421,375755,5057734,Closed,currency 1,-1335,0,-1152.0,-1152.0,NaN,0,195408.00,0.0,NaN,0.0,Consumer credit,-1139,NaN
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN


In [27]:
plus_day=[]
minus_day=[]
for i in br_df['DCE'].value_counts().head(10).index:
    if i>=0:
        plus_day.append(i)
    else:
        minus_day.append(i)
  
print(plus_day)
print(minus_day)

[0.0, 3.0, 1.0, 4.0]
[-7.0, -14.0, -10.0, -2.0, -1.0, -42.0]


In [28]:
pd.set_option('display.float_format',lambda x : '%.3f' % x)
(br_df['DCE']/365).describe()

count   1610875.000
mean          1.399
std          13.683
min        -115.233
25%          -3.118
50%          -0.904
75%           1.299
max          85.477
Name: DCE, dtype: float64

In [29]:
br_df['DCE'].replace({31199.0:np.nan},inplace=True)
br_df['DCE'].replace({-115.0:np.nan},inplace=True)
br_df['DCE'].max()

31198.0

In [30]:
br_df[(br_df['DEF']/-365)>=10]

,SK_ID_CURR,SIB,CA,CC,DC,CDO,DCE,DEF,ACMO,CCP,ACS,ACSD,ACSL,ACSO,CT,DCU,AA
198346,104463,6550986,Closed,currency 1,-2340,0,-1836.000,-42023.000,0.000,0,225000.000,0.000,225000.000,0.000,Credit card,-2196,NaN


In [31]:
br_df['DEF'].replace({-42023.0:np.nan},inplace=True)  #極端值處理

In [32]:
br_df['ACMO'].replace({115987185.0:np.nan},inplace=True) #極端值處理

In [33]:
abs_value = abs(br_df['DC'])
abs_value.describe()/365

count   4702.542
mean       3.129
std        2.179
min        0.000
25%        1.299
50%        2.704
75%        4.564
max        8.005
Name: DC, dtype: float64

In [34]:
def total_info(data):
    
    #＊要改columns檔名
    data_shape=pd.DataFrame([data.shape[0],data.shape[1],data.size],index=['Row','Column','Total Data'],columns=['bureau'])
    data_type_counts=pd.DataFrame(data.dtypes.value_counts(),columns=['bureau'])
    data_shape_df=data_shape._append(data_type_counts)
    print(data_shape_df)
    

    #a欄位型態
    data_type=data.dtypes
    
    #b差異值數量
    uniques_num=data.apply(lambda x:x.unique().shape[0],axis=0)
    
    #c空值比例
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    
    #abc
    columns_data=pd.concat([data_type,uniques_num,total, percent], axis=1, keys=['Type','Uniques_Num','Missing_Total', 'Missing_Percent'],sort=False)
    return columns_data

total_info(br_df)

              bureau
Row          1716428
Column            17
Total Data  29179276
float64            8
int64              6
object             3


,Type,Uniques_Num,Missing_Total,Missing_Percent
SK_ID_CURR,int64,305811,0,0.000
SIB,int64,1716428,0,0.000
CA,object,4,0,0.000
CC,object,4,0,0.000
DC,int64,2923,0,0.000
CDO,int64,942,0,0.000
DCE,float64,14095,106178,6.186
DEF,float64,2917,633654,36.917
ACMO,float64,68251,1124489,65.513
CCP,int64,10,0,0.000


In [35]:
pd.set_option('display.float_format',lambda x : '%.3f' % x)
br_df.describe()

,SK_ID_CURR,SIB,DC,CDO,DCE,DEF,ACMO,CCP,ACS,ACSD,ACSL,ACSO,DCU,AA
count,1716428.000,1716428.000,1716428.000,1716428.000,1610250.000,1082774.000,591939.000,1716428.000,1716415.000,1458759.000,1124648.000,1716428.000,1716428.000,489637.000
mean,278214.934,5924434.489,-1142.108,0.818,510.741,-1017.399,3629.480,0.006,354994.592,137085.120,6229.515,37.913,-593.748,15712.758
std,102938.558,532265.729,795.165,36.544,4995.115,712.923,140440.262,0.096,1149811.344,677401.131,45032.031,5937.650,720.747,325826.949
min,100001.000,5000000.000,-2922.000,0.000,-42060.000,-3042.000,0.000,0.000,0.000,-4705600.320,-586406.115,0.000,-41947.000,0.000
25%,188866.750,5463953.750,-1666.000,0.000,-1138.000,-1489.000,0.000,0.000,51300.000,0.000,0.000,0.000,-908.000,0.000
50%,278055.000,5926303.500,-987.000,0.000,-331.000,-897.000,0.000,0.000,125518.500,0.000,0.000,0.000,-395.000,0.000
75%,367426.000,6385681.250,-474.000,0.000,475.000,-425.000,0.000,0.000,315000.000,40153.500,0.000,0.000,-33.000,13500.000
max,456255.000,6843457.000,0.000,2792.000,31198.000,0.000,94812246.000,9.000,585000000.000,170100000.000,4705600.320,3756681.000,372.000,118453423.500


In [36]:
#br_df.drop(columns=['SIB', 'ACS', 'ACSD', 'ACSL', 'ACSO', 'CT', 'DCU', 'AA' ], inplace=True)
#AT=application_train.drop(application_train.loc[:, 'NAME_CONTRACT_TYPE':'AMT_REQ_CREDIT_BUREAU_YEAR'].columns, axis=1)
# br_df.groupby('SK_ID_CURR').mean()
# AB=AT.join(bdgm, on='SK_ID_CURR', how='left')
# AB.corr()

In [37]:
#欄位選取&遺失值處理

In [38]:
br_df.drop(columns=['SIB','CA','CC','CDO','ACMO','CCP','ACS', 'ACSD', 'ACSL', 'ACSO', 'CT', 'DCU', 'AA'], inplace=True)
br_df['DCE']=br_df['DCE'].fillna((br_df['DCE'].mean()))
br_df['DEF']=br_df['DEF'].fillna((br_df['DEF'].mean()))
bdgm=br_df.groupby('SK_ID_CURR').mean()

In [39]:
bureau_head = bdgm.rename(columns={'DC': 'BUREAU_DAYS_CREDIT','DCE': 'BUREAU_DAYS_CREDIT_ENDDATE','DEF': 'BUREAU_DAYS_ENDDATE_FACT'})
      
print(bureau_head)
bureau_head.isna().sum()/bureau_head.shape[0]

            BUREAU_DAYS_CREDIT  BUREAU_DAYS_CREDIT_ENDDATE  \
SK_ID_CURR                                                   
100001                -735.000                      82.429   
100002                -874.000                    -134.065   
100003               -1400.750                    -544.500   
100004                -867.000                    -488.500   
100005                -190.667                     439.333   
...                        ...                         ...   
456249               -1667.077                   -1098.251   
456250                -862.000                    1288.333   
456253                -867.500                     280.500   
456254               -1104.000                    -859.000   
456255               -1089.455                    3231.273   

            BUREAU_DAYS_ENDDATE_FACT  
SK_ID_CURR                            
100001                      -907.743  
100002                      -777.475  
100003                     -1077.350 

BUREAU_DAYS_CREDIT           0.000
BUREAU_DAYS_CREDIT_ENDDATE   0.000
BUREAU_DAYS_ENDDATE_FACT     0.000
dtype: float64

# Bureau tail


In [40]:
# 合併 bureau.csv 和 average.csv(貸款狀況評分)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

head = ['userID', 'SK_ID_BUREAU', 'CA', 'CC', 'DC',
        'CDO', 'DE', 'DF', 'MO', 'CP', 'CS', 'CD', 'CL', 'CO', 'CT', 'CU', 'annuity']
df = pd.read_csv('../../../../homecredit/bureau.csv') 
bal_df = pd.read_csv('../../../../homecredit/average.csv')
br_df = df.copy(deep=True)
br_df.columns = head
br_df.drop(columns=['CC', 'DC', 'CDO', 'DE', 'DF',
                   'MO', 'CP', 'CL', 'CO', 'CU'], inplace=True)

br_df = br_df.merge(bal_df, on='SK_ID_BUREAU', how='left')
print(br_df.head())

   userID  SK_ID_BUREAU      CA          CS         CD               CT  \
0  215354       5714462  Closed   91323.000      0.000  Consumer credit   
1  215354       5714463  Active  225000.000 171342.000      Credit card   
2  215354       5714464  Active  464323.500        NaN  Consumer credit   
3  215354       5714465  Active   90000.000        NaN      Credit card   
4  215354       5714466  Active 2700000.000        NaN  Consumer credit   

   annuity  average  
0      NaN      NaN  
1      NaN      NaN  
2      NaN      NaN  
3      NaN      NaN  
4      NaN      NaN  


In [41]:
# Credit Type 名稱冗長，重新命名
print(f"命名前一共 {len(list(set(br_df['CT'])))} 個類別")
renameDict = {'Unknown type of loan' : 'Unknown type', 'Unknown type': 'Unknown type','Loan for business development':'business development',
               'Loan for working capital replenishment':'working capital replenishment','Loan for purchase of shares (margin lending)':'purchase of shares', 
              'Consumer credit':'Consumer credit', 'Mortgage':'Mortgage', 'Real estate loan' : 'Real estate loan', 'Another type of loan':'Another type of loan',
                'Loan for the purchase of equipment' :'purchase of equipment','business development':'business development',
              'Cash loan' : 'Cash loan', 'Car loan' : 'Car loan', 'Credit card':'Credit card', 'Interbank credit':'Interbank credit', 'Mobile operator loan' : 'Mobile operator loan',
               'Cash loan (non-earmarked)':'Cash loan', 'Microloan':'Microloan', 'working capital replenishment':'working capital replenishment',
              }

br_df['CT'].replace(renameDict, inplace=True)
loanType = list(set(br_df['CT']))
print(f"命名後一共 {len(set(br_df['CT']))} 個類別")
print(loanType)

命名前一共 15 個類別
命名後一共 15 個類別
['Unknown type', 'business development', 'Credit card', 'Mortgage', 'Cash loan', 'purchase of equipment', 'Mobile operator loan', 'Microloan', 'Interbank credit', 'Car loan', 'working capital replenishment', 'purchase of shares', 'Consumer credit', 'Real estate loan', 'Another type of loan']


In [42]:
CTdf = br_df.iloc[:,2:-1].groupby('CT')
CTMeandf = pd.DataFrame(columns=['CT','CDmean', 'CSmean', 'annuitymean'])
tempList = []
for i in loanType:
    # print(i,'的各項平均')
    tempList.append(i)
    for j in ['CD', 'CS', 'annuity']:
        tempList.append(round(np.mean(CTdf.get_group(i)[j]),2))
        # print(f'{j}的平均值: {np.mean(CTdf.get_group(i)[j])}')
tempArray = np.array(tempList).reshape(-1,4)

for i in range(len(tempArray.tolist())):
    CTMeandf.loc[len(CTMeandf)] = tempArray.tolist()[i]

# 貸款類型(CT) 在各欄位的平均表現
for i in ['CT','CDmean', 'CSmean', 'annuitymean']:
    if i == 'CT':
        CTMeandf[i] = CTMeandf[i]
    else:
        CTMeandf[i]  = pd.to_numeric(CTMeandf[i] ,errors='coerce')

# 有貸款類型的值不能計算平均，在此補 0
CTMeandf = CTMeandf.fillna(0.0)

values = br_df['CT'].value_counts(dropna=False).keys().tolist()
counts = br_df['CT'].value_counts(dropna=False).tolist()
value_dict = dict(zip(values, counts))

tempList = []
for i in range(len(CTMeandf['CT'])):
    tempList.append(value_dict[CTMeandf['CT'][i]])

CTMeandf['samples'] = tempList

tempList = []
for i in range(len(CTMeandf['samples'])):
    n = round(CTMeandf['samples'][i]/(CTMeandf['samples'].sum()),3)
    tempList.append(n)
CTMeandf['percentage'] = tempList

In [43]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 將定義不清楚的貸款類別重新分類，eta: 3 mins
df.loc[df['CREDIT_TYPE']=='Another type of loan', 'CREDIT_TYPE'] = np.nan
df.loc[df['CREDIT_TYPE']=='Unknown type', 'CREDIT_TYPE'] = np.nan
df.loc[df['CREDIT_TYPE']=='Unknown type of loan', 'CREDIT_TYPE'] = np.nan

CA_ohe = pd.get_dummies(df['CREDIT_ACTIVE'],dtype=float)
CAType = list(set(df['CREDIT_ACTIVE']))

tempdf = pd.concat([df, CA_ohe], axis = 1)
# print(tempdf.isna().sum())
independentVariables = ['DAYS_CREDIT','CREDIT_DAY_OVERDUE','CNT_CREDIT_PROLONG','AMT_CREDIT_SUM_OVERDUE','DAYS_CREDIT_UPDATE', 'CREDIT_TYPE'] + CAType
tempdf = tempdf[independentVariables]

# 貸款類別轉成名目，並保留 NaN，以利後續 KNN 補值
le = LabelEncoder()
tempdf['CREDIT_TYPE'] = pd.Series(
    le.fit_transform(tempdf['CREDIT_TYPE'][tempdf['CREDIT_TYPE'].notnull()]),
    index=tempdf['CREDIT_TYPE'][tempdf['CREDIT_TYPE'].notnull()].index
)
# 使用 KNN 前先標準化，會導致
scaler = StandardScaler()
tempdf.iloc[:,:5] = scaler.fit_transform(tempdf.iloc[:,:5].to_numpy())

imputer = KNNImputer(n_neighbors=5)
tempdf = pd.DataFrame(imputer.fit_transform(tempdf),columns = tempdf.columns)
tempdf['CREDIT_TYPE'] = tempdf['CREDIT_TYPE'].astype(int)
# print(tempdf.isna().sum())

# 從名目轉回貸款類別
tempdf['CREDIT_TYPE'] = tempdf['CREDIT_TYPE'].astype(int)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)
inv_mapping = {v : ct for ct, v in le_name_mapping.items()}
tempdf['CREDIT_TYPE'] = tempdf['CREDIT_TYPE'].map(inv_mapping)
# print(tempdf['CREDIT_TYPE'].isna().sum())

br_df['CT'] = tempdf['CREDIT_TYPE']
br_df['CT'].replace(renameDict, inplace=True)

print(br_df['CT'].value_counts())

Consumer credit                  1252909
Credit card                       402277
Car loan                           27690
Mortgage                           18391
Microloan                          12415
business development                1983
working capital replenishment        471
Cash loan                            174
Interbank credit                      61
Real estate loan                      27
purchase of equipment                 20
purchase of shares                     9
Mobile operator loan                   1
Name: CT, dtype: int64


In [44]:
# 移除不到 1 % 的貸款類別(CT) (視為噪音)
rmCols = ['Mobile operator loan','Real estate loan','Cash loan','Interbank credit','purchase of equipment', 'purchase of shares']
for i in rmCols:
    br_df.drop(br_df.loc[br_df['CT']==i].index, inplace=True)
br_df = br_df.reset_index()
print(f"刪除後一共 {len(set(br_df['CT']))} 個類別")

刪除後一共 7 個類別


In [45]:
set(br_df['CT'])

{'Car loan',
 'Consumer credit',
 'Credit card',
 'Microloan',
 'Mortgage',
 'business development',
 'working capital replenishment'}

In [46]:
# 將債務金額(CD) 負值改成正值
br_df['CD'] = np.abs(br_df['CD'])
# print(br_df['CD'].min())

# 將極端值設為 NaN
for i in ['CD', 'CS', 'annuity']:
    if i == 'CD':
        br_df.loc[br_df[i] > 10000000 ,i] = np.nan
        br_df.loc[br_df[i] < 0, i] = 0
    elif i == 'CS':
        br_df.loc[br_df[i] > 20000000 ,i] = np.nan
    elif i == 'annuity':
        br_df.loc[br_df[i] > 10000000 ,i] = np.nan

print(br_df[['CD', 'CS', 'annuity', 'average']].describe())

               CD           CS     annuity    average
count 1457515.000  1715827.000  489537.000 694006.000
mean   127338.375   347941.918   13961.879      0.954
std    507222.157   818992.567   93346.522      0.251
min         0.000        0.000       0.000     -5.000
25%         0.000    51300.000       0.000      1.000
50%         0.000   125469.000       0.000      1.000
75%     39769.875   315000.000   13500.000      1.000
max   9999715.500 19998000.000 9979879.500      1.000


In [47]:
from sklearn.ensemble import RandomForestRegressor

# RF 預測 annuity NaN 並補值，eta: 4mins
# print(df.isna().any(axis = 0)) # 查看沒 NAN 的特徵有哪些，以選取可以比較的欄位 (不包括幣別)

Series = df['AMT_ANNUITY']
withAnn = df[pd.isnull(Series) == False]
withoutAnn = df[pd.isnull(Series)]
CA_ohe = pd.get_dummies(withAnn['CREDIT_ACTIVE'],dtype=float) # 4 類
CT_ohe = pd.get_dummies(withAnn['CREDIT_TYPE'],dtype=float) # 13 類
CTType = list(set(df['CREDIT_TYPE']))
CTType.remove('Interbank credit')
CTType.remove('Mobile operator loan')
CTType.remove(np.nan)
CAType = list(set(df['CREDIT_ACTIVE']))
withAnn = pd.concat([withAnn, CA_ohe, CT_ohe], axis = 1)
variables = ['DAYS_CREDIT','CREDIT_DAY_OVERDUE','CNT_CREDIT_PROLONG','AMT_CREDIT_SUM_OVERDUE','DAYS_CREDIT_UPDATE', 'AMT_ANNUITY'] + CAType + CTType # 23 cols
withAnn = withAnn[variables] # get_dum() 的 0 被轉成 NaN
withAnn = withAnn.fillna(0) 

CA_ohe = pd.get_dummies(withoutAnn['CREDIT_ACTIVE'],dtype=float)
CT_ohe = pd.get_dummies(withoutAnn['CREDIT_TYPE'],dtype=float)
withoutAnn = pd.concat([withoutAnn, CA_ohe, CT_ohe], axis = 1)
withoutAnn = withoutAnn[variables]
withoutAnn = withoutAnn.fillna(0)

independentVariables = ['DAYS_CREDIT','CREDIT_DAY_OVERDUE','CNT_CREDIT_PROLONG','AMT_CREDIT_SUM_OVERDUE','DAYS_CREDIT_UPDATE'] + CAType + CTType
X, y = withAnn[independentVariables], withAnn['AMT_ANNUITY']
X_pred = withoutAnn[independentVariables]

rfModel_ann = RandomForestRegressor()
rfModel_ann.fit(X, y)
y_pred = rfModel_ann.predict(X_pred)
withoutAnn['AMT_ANNUITY'] = y_pred.astype(int)
data = pd.concat([withAnn, withoutAnn])
data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)
print(data.shape)
print(data['AMT_ANNUITY'].isna().sum())

(1716428, 21)
0


In [48]:
CTMeandf['ratio'] = round(CTMeandf['CDmean'] / CTMeandf['CSmean'], 2)
# NaN 值數量
# CS                  312
# CD               258626
# 有 194 筆兩欄(CS、CD) 同時是 NaN 的貸款
allNan = br_df[br_df['CS'].isna() & br_df['CD'].isna()]
print(allNan.shape[0]) # 194

194


In [49]:
# 以該筆貸款的貸款類別(CT)平均額度使用率，同時補給CS, CD 的 nan 一個比例值
for i in allNan['SK_ID_BUREAU'].to_list():
    br_df.loc[br_df['SK_ID_BUREAU'] == i, 'CS'] = 0
    br_df.loc[br_df['SK_ID_BUREAU'] == i, 'CD'] = 0
br_df = pd.merge(br_df, CTMeandf[['CT','ratio']], on = 'CT')
br_df['CS'].fillna(br_df['CD'] / br_df['ratio'], inplace = True)
br_df['CD'].fillna(br_df['CS'] * br_df['ratio'], inplace = True)

In [50]:
# average NaN 補 0，即視同沒紀錄
br_df['average'] = br_df['average'].fillna(0.0)

# annuity NaN 以 random forest 補
br_df['annuity'] = data['AMT_ANNUITY']

# 刪掉先前計算用的欄位
del br_df['ratio']
del br_df['index']
print(br_df.isna().sum())
# br_df.to_csv('./data/fillna.csv')

userID          0
SK_ID_BUREAU    0
CA              0
CS              0
CD              0
CT              0
annuity         0
average         0
dtype: int64


In [51]:
# 賦予物理意義，eta: 4 mins
perUser = br_df.groupby(['userID'])
userList = list(set(br_df['userID']))

# 當月所需支付的年金總額
userSumAnn = []
# 信用額度使用率
userCD_CS_ratio = []
# 還款狀況
userStatus = []

# 因 CD 和 CS 都有 0 的值發生，所以產生出來的值須再補值一次
for i in userList:
    userSumAnn.append(np.sum(perUser.get_group(i)['annuity']))
    userCD_CS_ratio.append(np.sum(perUser.get_group(i)['CD'])/np.sum(perUser.get_group(i)['CS']))
    userStatus.append(np.average(perUser.get_group(i)['average']))

C:\Users\User\AppData\Local\Temp\ipykernel_9964\1882926969.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  userCD_CS_ratio.append(np.sum(perUser.get_group(i)['CD'])/np.sum(perUser.get_group(i)['CS']))
C:\Users\User\AppData\Local\Temp\ipykernel_9964\1882926969.py:15: RuntimeWarning: invalid value encountered in double_scalars
  userCD_CS_ratio.append(np.sum(perUser.get_group(i)['CD'])/np.sum(perUser.get_group(i)['CS']))


In [52]:
# 最終每位 User 的 df
final_df = pd.DataFrame(
     {'SK_ID_CURR': userList,'CDCS_ratio': userCD_CS_ratio, 'bureau_annuity_mean': userSumAnn, 'bereau_balance_status': userStatus})
# print(final_df.iloc[:, 1:].describe())

# 有 1224 人沒有信用額度使用率，猜測是之前為小白，現在卻開始貸款，所以無聯徵紀錄
# 而完全沒在聯徵資料出現的貸款者，猜測其當前仍然是小白
print('--- check isna() ---')
print(final_df.isna().sum())

# 將信用額度使用率 nan 補 0 ，即此人的過往聯徵是小白
final_df['CDCS_ratio'].fillna(0.0, inplace = True)
final_df[np.isinf(final_df['CDCS_ratio'])] = 0
print('--- Final version check isna() ---')
print(final_df.isna().sum())
# final_df.to_csv('./data/bureau_final.csv', index = False)

--- check isna() ---
SK_ID_CURR                  0
CDCS_ratio               1218
bureau_annuity_mean         0
bereau_balance_status       0
dtype: int64
--- Final version check isna() ---
SK_ID_CURR               0
CDCS_ratio               0
bureau_annuity_mean      0
bereau_balance_status    0
dtype: int64


被歸類在極端值的貸款人

最終表呈現

In [53]:
print(final_df.head())
print()
print(f'筆數: {final_df.shape[0]}')
bureau_tail = final_df
bureau_tail

   SK_ID_CURR  CDCS_ratio  bureau_annuity_mean  bereau_balance_status
0      100001       0.411           237469.500                  0.978
1      100002       0.349           100580.000                  0.402
2      100003       0.000            70437.000                  0.000
3      100004       0.000            54004.500                  0.000
4      100005       0.865            42074.000                  1.000

筆數: 305794


,SK_ID_CURR,CDCS_ratio,bureau_annuity_mean,bereau_balance_status
0,100001,0.411,237469.500,0.978
1,100002,0.349,100580.000,0.402
2,100003,0.000,70437.000,0.000
3,100004,0.000,54004.500,0.000
4,100005,0.865,42074.000,1.000
...,...,...,...,...
305789,456249,0.173,167521.000,0.000
305790,456250,0.723,21968.000,1.000
305791,456253,0.453,11269.000,1.000
305792,456254,0.000,7737.000,1.000


### application_train : application_train
### previous_application: prev_cash_loans, prev_consumer_loans, prev_revolving_loans
### credit_card:credit_card_balance
### instalments: instalments
### bureau: bureau_head, bureau_tail



In [54]:
dfs = [prev_cash_loans, prev_consumer_loans, prev_revolving_loans, credit_card_balance, instalments, bureau_head, bureau_tail]

train_all_v1 = application_train  

for df in dfs:
    train_all_v1 = pd.merge(train_all_v1, df, on="SK_ID_CURR", how="left")

print(train_all_v1.columns)

train_all_v1


C:\Users\User\AppData\Local\Temp\ipykernel_9964\1120613046.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'NAME_CONTRACT_TYPE_x'} in the result is deprecated and will raise a MergeError in a future version.
  train_all_v1 = pd.merge(train_all_v1, df, on="SK_ID_CURR", how="left")


Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE_x', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'AGE', 'JOB_TENURE',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', '365243DAYS_EMPLOYED',
       'NAME_CONTRACT_TYPE_y', 'SK_ID_PREV_x', 'AMT_ANNUITY_CASH_LOANS_MEAN',
       'AMT_APPLICATION_CA

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,creditcard_low,instalments_overdue_DAY_mean,instalments_overdue_AMT_mean,instalments_PERIOD,BUREAU_DAYS_CREDIT,BUREAU_DAYS_CREDIT_ENDDATE,BUREAU_DAYS_ENDDATE_FACT,CDCS_ratio,bureau_annuity_mean,bereau_balance_status
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.500,24700.500,...,NaN,-20.421,0.000,19.000,-874.000,-134.065,-777.475,0.349,100580.000,0.402
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.500,35698.500,...,NaN,-7.160,0.000,8.333,-1400.750,-544.500,-1077.350,0.000,70437.000,0.000
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.000,6750.000,...,NaN,-7.667,0.000,3.000,-867.000,-488.500,-532.500,0.000,54004.500,0.000
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.500,29686.500,...,0.000,-19.375,0.000,5.333,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.000,21865.500,...,NaN,-3.636,-452.384,12.600,-1149.000,-783.000,-783.000,0.000,16645.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.000,254700.000,27558.000,...,NaN,-36.286,0.000,7.000,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.000,269550.000,12001.500,...,NaN,-2.833,0.000,6.000,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.000,677664.000,29979.000,...,NaN,-14.500,-283.792,7.333,-867.500,280.500,-905.700,0.453,11269.000,1.000
307509,456254,1,Cash loans,F,N,Y,0,171000.000,370107.000,20205.000,...,NaN,-19.000,0.000,9.500,-1104.000,-859.000,-859.000,0.000,7737.000,1.000


In [55]:
dfs = [application_train ,prev_cash_loans, prev_consumer_loans, prev_revolving_loans, credit_card_balance, instalments, bureau_head, bureau_tail]

name =  ['application_train' ,'prev_cash_loans', 'prev_consumer_loans', 'prev_revolving_loans', 'credit_card_balance', 'instalments', 'bureau_head', 'bureau_tail']

i=0
for df in dfs:
    print(f'{name[i]}:{df.shape}')
    i += 1
i=0
for df in dfs:
    na_percentages = df.isna().mean() * 100
    na_columns = na_percentages[na_percentages > 0] 
    print(f'{name[i]}:\n{na_columns}')
    i +=1
print('-------------------------')
print('train_all_v1.isna%:')
na_percentages = train_all_v1.isna().mean() * 100
na_columns = na_percentages[na_percentages > 0]  # 选择缺失值比例大于 0 的列
print(na_columns)

application_train:(307511, 39)
prev_cash_loans:(200825, 7)
prev_consumer_loans:(313316, 7)
prev_revolving_loans:(123445, 7)
credit_card_balance:(103558, 4)
instalments:(339587, 4)
bureau_head:(305811, 3)
bureau_tail:(305794, 4)
application_train:
Series([], dtype: float64)
prev_cash_loans:
Series([], dtype: float64)
prev_consumer_loans:
Series([], dtype: float64)
prev_revolving_loans:
Series([], dtype: float64)
credit_card_balance:
Series([], dtype: float64)
instalments:
Series([], dtype: float64)
bureau_head:
Series([], dtype: float64)
bureau_tail:
Series([], dtype: float64)
-------------------------
train_all_v1.isna%:
NAME_CONTRACT_TYPE_y                   44.268
SK_ID_PREV_x                           44.268
AMT_ANNUITY_CASH_LOANS_MEAN            44.268
AMT_APPLICATION_CASH_LOANS_MEAN        44.268
AMT_CREDIT_CASH_LOANS_MEAN             44.268
AMT_GOODS_PRICE_CASH_LOANS_MEAN        44.268
NAME_CONTRACT_TYPE_x                   12.516
SK_ID_PREV_y                           12.516
AMT

In [60]:
train_all_v1.drop(columns=['NAME_CONTRACT_TYPE_x', 'NAME_CONTRACT_TYPE_y', 'SK_ID_PREV_x', 'SK_ID_PREV_y', 'NAME_CONTRACT_TYPE_y',
                   'SK_ID_PREV'], inplace=True)
train_all_v1.columns

Index(['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
       'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
       'REGION_POPULATION_RELATIVE', 'AGE', 'JOB_TENURE', 'DAYS_REGISTRATION',
       'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
       '365243DAYS_EMPLOYED', 'AMT_ANNUITY_CASH_LOANS_MEAN',
       'AMT_APPLICATION_CASH_LOANS_MEAN', 'AMT_CREDIT_CASH_LOANS_MEAN',
       'AMT_GOODS_PRICE_C

In [63]:
na_percentages = train_all_v1.isna().mean() * 100
na_columns = na_percentages[na_percentages > 0]  # 选择缺失值比例大于 0 的列
print(na_columns)

Series([], dtype: float64)


In [62]:
columns_to_fillna = [
    'AMT_ANNUITY_CASH_LOANS_MEAN', 'AMT_APPLICATION_CASH_LOANS_MEAN',
    'AMT_CREDIT_CASH_LOANS_MEAN', 'AMT_GOODS_PRICE_CASH_LOANS_MEAN',
    'AMT_ANNUITY_CONSUMER_LOANS_MEAN', 'AMT_APPLICATION_CONSUMER_LOANS_MEAN',
    'AMT_CREDIT_CONSUMER_LOANS_MEAN', 'AMT_GOODS_PRICE_CONSUMER_LOANS_MEAN',
    'AMT_ANNUITY_REVOLVING_LOANS_MEAN', 'AMT_APPLICATION_REVOLVING_LOANS_MEAN',
    'AMT_CREDIT_REVOLVING_LOANS_MEAN', 'AMT_GOODS_PRICE_REVOLVING_LOANS_MEAN',
    'creditcard_high', 'creditcard_normal', 'creditcard_low',
    'instalments_overdue_DAY_mean', 'instalments_overdue_AMT_mean',
    'instalments_PERIOD', 'BUREAU_DAYS_CREDIT', 'BUREAU_DAYS_CREDIT_ENDDATE',
    'BUREAU_DAYS_ENDDATE_FACT', 'CDCS_ratio', 'bureau_annuity_mean',
    'bereau_balance_status'
]

#合併後的ALL缺失值補0
train_all_v1[columns_to_fillna] = train_all_v1[columns_to_fillna].fillna(0)


In [66]:
train_all_v1.to_csv('train_all_v1.csv')